# Reflection and Blogpost Writing

## Setup

In [1]:
import os
api_key = os.environ["MISTRAL_API_KEY"]
llm_config = { "model": "mistral-large-latest", "api_key": api_key, "api_type": "mistral"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task,"role": "user"}])

In [5]:
print(reply)

{'content': "### Hope AI: The Future of Healthcare\n\nImagine a world where early disease detection is as simple as taking a selfie. Welcome to the future of healthcare with Hope AI. This revolutionary app combines the power of artificial intelligence with medical expertise to detect pancreatic cancer and other critical conditions at their earliest stages.\n\nWith Hope AI, users can upload a simple photo, and the app's advanced algorithms analyze subtle facial changes that may indicate underlying health issues. This groundbreaking technology aims to make early detection accessible and convenient, potentially saving countless lives.\n\nEmbrace the future of healthcare with Hope AI—where hope meets innovation.", 'refusal': None, 'role': 'assistant', 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': None}


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Title: Hope AI: The Beacon of Possibilities**

Imagine an AI that doesn't just learn, but hopes. Meet Hope AI, a groundbreaking concept that combines artificial intelligence with the very human emotion of hope. Unlike traditional AI, Hope AI isn't just about data and patterns; it's about possibilities. It dreams of better outcomes, explores unlikely solutions, and persists when others might quit. In healthcare, it could mean finding cures previously thought impossible. In climate change, it could mean discovering innovative solutions. Hope AI isn't just about what is; it's about what could be. It's more than a tool—it's an inspiration.

--------------------------------------------------------------------------------
Critic (to Writer):


## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO_Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal_Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics_Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta_Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

**Title: Hope AI: The Beacon of Possibilities**

Imagine an AI that doesn't just learn, but hopes. Meet Hope AI, a groundbreaking concept that combines artificial intelligence with the very human emotion of hope. Unlike traditional AI, Hope AI isn't just about data and patterns; it's about possibilities. It dreams of better outcomes, explores unlikely solutions, and persists when others might quit. In healthcare, it could mean finding cures previously thought impossible. In climate change, it could mean discovering innovative solutions. Hope AI isn't just about what is; it's about what could be. It's more than a tool—it's an inspiration.

--------------------------------------------------------------------------------

*******************

## Get the summary

In [15]:
print(res.summary)

**Title: Exploring Possibilities with Innovative AI**

Imagine an AI that ventures beyond current limitations. Introducing Hope AI, a pioneering concept that combines artificial intelligence with an unyielding drive to explore new possibilities. This innovative AI isn't just about data and patterns; it's about envisioning better outcomes and seeking unlikely solutions.

**Hope AI in Healthcare**

In healthcare, Hope AI could aid in exploring potential cures and treatments previously thought challenging. By analyzing vast amounts of data and predicting trends, it could accelerate medical breakthroughs and enhance patient care.

**Hope AI for Climate Change**

In the realm of climate change, Hope AI could contribute to discovering innovative solutions. By modeling complex environmental systems and simulating various scenarios, it could help us better understand and address global warming and its impacts.

**A Beacon of Innovation**

Hope AI isn't just a tool; it's an inspiration. It remi